In [13]:
import logging
import time
import random
from datetime import datetime

n_campi = 'CPMR'

data_e_hora_atuais = datetime.now()
data_e_hora = data_e_hora_atuais.strftime('%Y-%m-%d_%H-%M')

logger = logging.getLogger('consolidação')
logger.setLevel(logging.DEBUG)

fHandler = logging.FileHandler(f'logs/{data_e_hora_atuais}_consolidacao.log', mode='w')
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

fHandler.setFormatter(formatter)
logger.addHandler(fHandler)

# logging.basicConfig(level=logging.DEBUG, filename='app.log', filemode='x', format='%(asctime)s - %(levelname)s - %(message)s')
logger.info(f'Iniciando consolidação campus {n_campi}')
# logger.debug('debug message')
# logger.warning('warn message')

In [14]:
import pandas as pd

logger.info(f'carregando palnilha de levantamento fisico campus {n_campi}')
levantamento = pd.read_excel('consolidacao/levantamento/2023_levantamento_CPMR.xlsx')

logger.info(f'carregando palnilha de estoque suap campus {n_campi}')
suap = pd.read_excel('consolidacao/suap/2023-10-10_08-37_CPMR_relatorio_almox.xlsx')

logger.info(f'tratamdp dados da planilha de levantameto fisico {n_campi}')

# removendo itens nulos e deixando apenas os retificados
retificacao = levantamento[levantamento['Retificação'].str.contains('Retificação').fillna(False)]


# lista de itens vencidos
# vencido = levantamento[levantamento['Material Vencido - MV'].str.contains('x').fillna(False)]
# lista de itens não identificados
# nao_identificado = levantamento[levantamento['Sem identificação - SI'].str.contains('x').fillna(False)]

# print(levantamento.info())
# print(suap.info())

# planilha contendo apenas itens válidos
levantamento_ret = levantamento.drop_duplicates(subset=['Código'], keep='last')
levantamento_ret = levantamento_ret.rename(columns={'Código': 'codigo'})

# elemento despesa do levantamento
ed_lev = levantamento_ret['codigo'].str[3:5]


# adicionando ed ao data frame geral
# levantamento_ret['ED'] = ed_lev

logger.info(f'removendo dados duplicados')
# lista de servidores
servidores = levantamento['Endereço de e-mail'].drop_duplicates(keep='last')
# reindexando uutilizando list comprehension
servidores.index = [x for x in range(0, len(servidores))]

logger.info(f'cruzando dados entre levantamento fisico e suap')

levantamento_cross = levantamento_ret.drop(columns=['Material de pouca movimentação - PM', 'Material Vencido - MV',
                                                 'Material próximo ao vencimento - PV', 'Material danificado ou avariado - MD',
                                                 'Material Obsoleto - MO', 'Sem identificação - SI', 'Valor unitário',
                                                 'Campus', 'Retificação', 'Carimbo de data/hora'])
levantamento_cross = levantamento_cross.rename(columns={'Endereço de e-mail': 'servidor', 'Nome': 'descricao_lev',
                                                        'Quantidade': 'qtd_lev'})
levantamento_cross = levantamento_cross[['codigo', 'descricao_lev', 'qtd_lev', 'servidor']]

suap_cross = suap.rename(columns={'descricao': 'descricao_suap', 'qtd': 'qtd_suap'})
suap_cross = suap_cross.drop(columns=['vlr_total', 'vlr_unt'])
# display(suap_cross)
# display(levantamento_cross)

logger.info(f'consolidando dados')

logger.info(f'exportando planilha de consolidação')
consolidacao = pd.merge(suap_cross, levantamento_cross, on='codigo', how='outer', indicator=True)
consolidacao.to_excel(f'consolidacao/{n_campi}_consolidacao.xlsx', index=False)

logger.info(f'consolidação campus {n_campi} finalizada')

In [15]:
logger.info(f'montando planilha de ajustes ')

consolidacao['diferenca'] = consolidacao['qtd_suap'] - consolidacao['qtd_lev']
display(consolidacao)

pedidos = consolidacao[consolidacao['_merge'].str.contains('both')]
pedidos = pedidos[['codigo','ed' ,'descricao_suap', 'qtd_suap', 'qtd_lev', 'diferenca']]

acao = []
for valor in pedidos['diferenca']:
    
    if valor > 0:
        baixa = "pedido no suap"
        acao.append(baixa)
    elif valor < 0:
        baixa = "saida sem pedido"
        acao.append(baixa)
    else:
        baixa = 'adequado'
        acao.append(baixa)

logger.info(f'exportando planilha de ajustes ')

pedidos['acao'] = acao
pedidos.to_excel(f'consolidacao/pedidos/{n_campi}_pedidos.xlsx', index=False)
display(pedidos)

,codigo,descricao_suap,und,qtd_suap,ed,descricao_lev,qtd_lev,servidor,_merge,diferenca
0,30.07.1462,Açucar cristal 1KG,UN,64,07 - GENEROS DE ALIMENTACAO,Açucar cristal 1KG,48,edmar.filho@reitoria.ifpe.edu.br,both,16
1,30.14.156,BRAÇADEIRA ELASTICA DE CAPITÃO.,UN,1,14 - MATERIAL EDUCATIVO E ESPORTIVO,BRAÇADEIRA ELASTICA DE CAPITÃO.,1,taciana.moura@palmares.ifpe.edu.br,both,0
2,30.14.23,BOLA DE VOLEIBOL P/ QUADRA,UN,1,14 - MATERIAL EDUCATIVO E ESPORTIVO,BOLA DE VOLEIBOL P/ QUADRA,1,taciana.moura@palmares.ifpe.edu.br,both,0
3,30.14.311,"KIT CARTÃO ARBITRO, FUTEBOL DE CAMPO, OFICIAL,...",UN,1,14 - MATERIAL EDUCATIVO E ESPORTIVO,"KIT CARTÃO ARBITRO, FUTEBOL DE CAMPO, OFICIAL,...",1,taciana.moura@palmares.ifpe.edu.br,both,0
4,30.14.475,Rede oficial de vôlei de areia. Confeccionada ...,UN,1,14 - MATERIAL EDUCATIVO E ESPORTIVO,Rede oficial de vôlei de areia. Confeccionada ...,1,taciana.moura@palmares.ifpe.edu.br,both,0
...,...,...,...,...,...,...,...,...,...,...
355,30.42.974,"CHAVE FENDA, MATERIAL HASTE AÇO VANÁDIO TREFIL...",UN,14,42 - FERRAMENTAS,"CHAVE FENDA, MATERIAL HASTE AÇO VANÁDIO TREFIL...",14,claudeandela.cavalcanti@reitoria.ifpe.edu.br,both,0
356,30.42.978,"CHAVE DE PHILLIPS, HASTE EM AÇO VANÁDIO, CABO ...",UN,5,42 - FERRAMENTAS,"CHAVE DE PHILLIPS, HASTE EM AÇO VANÁDIO, CABO ...",5,claudeandela.cavalcanti@reitoria.ifpe.edu.br,both,0
357,30.42.990,"SUPORTE PARA FERRO DE SOLDA, BASE EM CHAPA MET...",UN,10,42 - FERRAMENTAS,"SUPORTE PARA FERRO DE SOLDA, BASE EM CHAPA MET...",11,claudeandela.cavalcanti@reitoria.ifpe.edu.br,both,-1
358,30.44.48,"PROTETOR CRACHÁ, NOME BOLSA DE CRACHA",UN,39,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS,"PROTETOR CRACHÁ, NOME BOLSA DE CRACHA",39,taciana.moura@palmares.ifpe.edu.br,both,0


,codigo,ed,descricao_suap,qtd_suap,qtd_lev,diferenca,acao
0,30.07.1462,07 - GENEROS DE ALIMENTACAO,Açucar cristal 1KG,64,48,16,pedido no suap
1,30.14.156,14 - MATERIAL EDUCATIVO E ESPORTIVO,BRAÇADEIRA ELASTICA DE CAPITÃO.,1,1,0,adequado
2,30.14.23,14 - MATERIAL EDUCATIVO E ESPORTIVO,BOLA DE VOLEIBOL P/ QUADRA,1,1,0,adequado
3,30.14.311,14 - MATERIAL EDUCATIVO E ESPORTIVO,"KIT CARTÃO ARBITRO, FUTEBOL DE CAMPO, OFICIAL,...",1,1,0,adequado
4,30.14.475,14 - MATERIAL EDUCATIVO E ESPORTIVO,Rede oficial de vôlei de areia. Confeccionada ...,1,1,0,adequado
...,...,...,...,...,...,...,...
355,30.42.974,42 - FERRAMENTAS,"CHAVE FENDA, MATERIAL HASTE AÇO VANÁDIO TREFIL...",14,14,0,adequado
356,30.42.978,42 - FERRAMENTAS,"CHAVE DE PHILLIPS, HASTE EM AÇO VANÁDIO, CABO ...",5,5,0,adequado
357,30.42.990,42 - FERRAMENTAS,"SUPORTE PARA FERRO DE SOLDA, BASE EM CHAPA MET...",10,11,-1,saida sem pedido
358,30.44.48,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS,"PROTETOR CRACHÁ, NOME BOLSA DE CRACHA",39,39,0,adequado
